# Widgets

In [ ]:
# hide
import sys
sys.path.append("..")

In [ ]:
# export
# default_exp widgets
from faimed3d.basics import *
from faimed3d.data import *
from faimed3d.augment import *
from faimed3d.learner import *
from fastai.basics import *

In [ ]:
# export
class HookActivation():
    def __init__(self, target_layer):
        "Initialize a Pytorch hook using `hook_activation` function."
        self.hook = target_layer.register_forward_hook(self.hook_activation) 
        
    def hook_activation(self, target_layer, activ_in, activ_out): 
        "Create a copy of the layer output activations and save in `self.stored`."
        self.stored = activ_out.detach().clone()
        
    def __enter__(self, *args): 
        return self
    
    def __exit__(self, *args): 
        self.hook.remove()

        
class HookGradient():
    def __init__(self, target_layer):
        "Initialize a Pytorch hook using `hook_gradient` function."
        self.hook = target_layer.register_backward_hook(self.hook_gradient)   
        
    def hook_gradient(self, target_layer, gradient_in, gradient_out): 
        "Create a copy of the layer output gradients and save in `self.stored`."
        self.stored = gradient_out[0].detach().clone()
        
    def __enter__(self, *args): 
        return self

    def __exit__(self, *args): 
        self.hook.remove()

In [ ]:
from fastai.learner import Learner # for compatibility with show_docs

In [ ]:
#export
@patch
def grad_cam(learn:Learner, img, target_class=None, target_layer=None, *args, **kwargs):
    if target_layer is None: target_layer = learn.model[-2][4]
    pred = learn.predict(img)
    print('actual:',pred[0], '\t predicted:', pred[1].numpy(), '\t class probabilities:', *pred[2].numpy())
    if target_class is None: target_class = pred[1].numpy()
    # get images    
    x,  = first(dls.test_dl([img])) 
    x = torch.stack((x, )*3, 1)
    x_img = TensorDicom3D(dls.train.decode((x,))[0][0])
    # create cam
    with HookGradient(target_layer) as hookg:
        with HookActivation(target_layer) as hook:
            output = learn.model.eval()(x.cuda())
            act = hook.stored
        output[0, target_class].backward()
        grad = hookg.stored
        p0, p1 = output.cpu().detach()[0]
        
    w = grad[0].mean(dim=(1,2), keepdim=True)
    gradcam_map = (w * act[0]).sum(0).detach().cpu()
    gradcam_map = Resize3D(x_img[0].shape)(TensorDicom3D(gradcam_map))
    # plot
    x_img[0].show(*args, **kwargs)
    gradcam_map.clamp(min=0).show(cmap='inferno', alpha = 0.5, add_to_existing = True,*args, **kwargs)